# Лабораторная работа 2

Решить задачу классификации датасета MNIST используя MLP из scikitlearn и используя CNN (по типу LeNet) c пакетом PyTorch. \
Сравнить результаты по метрикам, сделать обоснованные выводы

## MLP

In [1]:
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

Загрузка датасета

In [2]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

Нормализация

In [3]:
X = X.astype('float32') / 255.0
y = y.astype(int)

Разделение на обучающий и тестовый набор

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

Обучение

In [5]:
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    max_iter=20,
    activation='relu',
    solver='adam',
    batch_size=256,
    random_state=42
)

start_time_mlp = time.time()
mlp.fit(X_train, y_train)
end_time_mlp = time.time()
print(f"Time: {end_time_mlp - start_time_mlp} sec.")

Time: 17.673134803771973 sec.


/home/alice/.pyenv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
y_pred = mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

Accuracy: 0.9755238095238096
              precision    recall  f1-score   support

           0     0.9873    0.9830    0.9851      2058
           1     0.9857    0.9898    0.9878      2364
           2     0.9694    0.9808    0.9751      2133
           3     0.9731    0.9632    0.9681      2176
           4     0.9733    0.9788    0.9760      1936
           5     0.9693    0.9723    0.9708      1915
           6     0.9732    0.9923    0.9827      2088
           7     0.9811    0.9715    0.9763      2248
           8     0.9693    0.9659    0.9676      1992
           9     0.9713    0.9560    0.9636      2090

    accuracy                         0.9755     21000
   macro avg     0.9753    0.9754    0.9753     21000
weighted avg     0.9755    0.9755    0.9755     21000



Особенности MLP:
- Простая реализация
- Быстро обучается
- Игнорирует геометрию изображения
- Хуже различает похожие цифры

## CNN

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

Датасет

In [8]:
X_train_torch = torch.tensor(X_train).reshape(-1, 1, 28, 28)
X_test_torch = torch.tensor(X_test).reshape(-1, 1, 28, 28)
y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset = TensorDataset(X_test_torch, y_test_torch)

BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


Сеть

In [10]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1-й сверточный слой (1 канал -> 6 выходных каналов)
        # Вход: 1x28x28. После Conv(5x5, stride=1): 6x24x24
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        # После MaxPool(2x2, stride=2): 6x12x12
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 2-й сверточный слой (6 каналов -> 16 выходных каналов)
        # Вход: 6x12x12. После Conv(5x5, stride=1): 16x8x8
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        # После MaxPool(2x2, stride=2): 16x4x4
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Полносвязные слои
        # Входной размер: 16 каналов * 4 * 4 = 256
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10) # 10 классов (цифры от 0 до 9)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # Выравнивание тензора (flatten) для полносвязных слоев
        x = x.view(-1, 16 * 4 * 4)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x


model = LeNet().to(device)

Обучение

In [11]:
def train_cnn(model, loader, criterion, optimizer, device, num_epochs):
    model.train()
    start_time = time.time()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad() # Обнуляем градиенты

            outputs = model(inputs) # Прямой проход
            loss = criterion(outputs, labels) # Вычисление ошибки
            loss.backward() # Обратный проход
            optimizer.step() # Обновление весов

            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(loader.dataset)
        print("Epoch", epoch+1, "/", num_epochs, "Loss:", epoch_loss)
    
    end_time = time.time()
    return end_time - start_time

In [12]:
CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = optim.Adam(model.parameters(), lr=0.001)
NUM_EPOCHS = 20
training_time_cnn = train_cnn(model, train_loader, CRITERION, OPTIMIZER, device, NUM_EPOCHS)

Epoch 1 / 20 Loss: 0.32007292953376865
Epoch 2 / 20 Loss: 0.08946778539370517
Epoch 3 / 20 Loss: 0.06338726250431975
Epoch 4 / 20 Loss: 0.050906627253154105
Epoch 5 / 20 Loss: 0.04236909556578921
Epoch 6 / 20 Loss: 0.0360330908167347
Epoch 7 / 20 Loss: 0.0303836109467535
Epoch 8 / 20 Loss: 0.026606045364877397
Epoch 9 / 20 Loss: 0.02337437690205264
Epoch 10 / 20 Loss: 0.020521667275839123
Epoch 11 / 20 Loss: 0.017740705186900282
Epoch 12 / 20 Loss: 0.017130295392672284
Epoch 13 / 20 Loss: 0.014796193687748925
Epoch 14 / 20 Loss: 0.012822255553876059
Epoch 15 / 20 Loss: 0.011531907941849086
Epoch 16 / 20 Loss: 0.011323884834875163
Epoch 17 / 20 Loss: 0.010408526655409797
Epoch 18 / 20 Loss: 0.011482389807268711
Epoch 19 / 20 Loss: 0.008214721758592142
Epoch 20 / 20 Loss: 0.005623247039574436


In [13]:
def evaluate_cnn(model, loader, device):
    model.eval()
    y_true = []
    y_pred = []
    correct = 0
    total = 0
    with torch.no_grad(): # Отключение вычисления градиентов для экономии памяти
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = correct / total
    return accuracy, y_true, y_pred

# Оценка
accuracy_cnn, y_test_true_cnn, y_pred_cnn = evaluate_cnn(model, test_loader, device)

print(f"Time: {training_time_cnn:.2f} sec.")
print(f"Accuracy: {accuracy_cnn}")
print(classification_report(y_test_true_cnn, y_pred_cnn, digits=4))

Time: 68.40 sec.
Accuracy: 0.9870476190476191
              precision    recall  f1-score   support

           0     0.9923    0.9961    0.9942      2058
           1     0.9962    0.9966    0.9964      2364
           2     0.9827    0.9869    0.9848      2133
           3     0.9875    0.9807    0.9841      2176
           4     0.9932    0.9788    0.9860      1936
           5     0.9900    0.9781    0.9840      1915
           6     0.9848    0.9933    0.9890      2088
           7     0.9950    0.9800    0.9874      2248
           8     0.9811    0.9880    0.9845      1992
           9     0.9673    0.9904    0.9787      2090

    accuracy                         0.9870     21000
   macro avg     0.9870    0.9869    0.9869     21000
weighted avg     0.9871    0.9870    0.9871     21000



## Вывод
CNN (LeNet-подобная) демонстрирует значительно более высокую точность (98-99%) по сравнению с MLP (96-97%). \
Причина: CNN специально разработана для обработки изображений. Сверточные слои (nn.Conv2d) способны автоматически изучать пространственные и иерархические признаки (границы, углы, формы) из исходных пикселей. \
MLP обрабатывает входные данные как плоский вектор (784 признака), теряя информацию о пространственной близости между пикселями. \
Для MLP пиксель в верхнем левом углу так же "связан" с пикселем в нижнем правом углу, как и с соседним пикселем, что неверно для изображений.

CNN использует общие веса (share weights) в пределах одного сверточного фильтра, что значительно уменьшает количество параметров по сравнению с полностью связанной (полносвязной) сетью с таким же количеством нейронов. Это делает CNN более эффективной и менее склонной к переобучению при работе с изображениями.

Сложность реализации и фреймворки:
- MLP с scikit-learn — очень простая реализация, идеальна для быстрого прототипирования.
- CNN с PyTorch — требует больше кода (определение класса сети, циклы обучения с батчами, оптимизатор, функция потерь), но дает полный контроль над архитектурой и процессом обучения, что необходимо для более сложных задач.